# KAZNIVA DEJANJA V SLOVENIJI

## 1. Predstavitev in predprocesiranje podatkov

Podatki so zbrani v 10 csv datotekah, pri čemer vsaka predstavlja po eno leto (od 2009 do 2019). Podatke iz teh datotek najprej zberemo v eno datoteko, "kd.csv". Spremenimo tudi nekatere vrednosti.
Ker se zaporedne številke resetirajo vsako leto, jih spremenimo tako, da se začnejo z letom, ko je bil podatek zapisan v tabelo (npr. ID 61193 iz leta 2019 postane 2019061193).
Poleg tega spremenimo vrednosti kot so ura ter dnevi v tednu, iz podatke "Object" v podatke tipa "int64". 
Smiselno spremenimo tudi vrednosti DA/NE (npr. ali je šlo za poskus) v podatke tipa bool (true/false).

In [10]:
def parseUraStoritve(series):
    ure = []
    for i, val in series.items():
        ure.append(int(val.split(":")[0]))
    return pd.Series(ure, dtype="int")

df_set = False
for year in range(2009, 2020):
    df_temp = pd.read_csv("data/kd" + str(year) + ".csv",
                          delimiter=";",
                          low_memory=False,
                          dtype={"DatumStoritve": "string",
                                 "MesecStoritve": "string",
                                 "UraStoritve": "string"})
    # ZaporednaStevilkaKD
    df_temp.iloc[:, 0] = df_temp.iloc[:, 0] + year * 1000_000

    # MesecStoritve
    # UraStoritve
    df_temp.iloc[:, 2] = parseUraStoritve(df_temp.iloc[:, 2])
    # DanVTednu
    df_temp.iloc[:, 3] = df_temp.iloc[:, 3].replace(["PONEDELJEK",
                                                     "TOREK",
                                                     "SREDA",
                                                     "�ETRTEK",
                                                     "PETEK",
                                                     "SOBOTA",
                                                     "NEDELJA"], [1, 2, 3, 4, 5, 6, 7]).astype(int)
    # PUStoritveKD
    # Povratnik
    df_temp.iloc[:, 5] = df_temp.iloc[:, 5].replace(["DA", "NE"], [1, 0]).astype("bool")
    # OpisKD
    # PoglavjeKD
    # GospodarskiKriminal
    df_temp.iloc[:, 8] = df_temp.iloc[:, 8].replace(["GOSPODARSKA", "SPLO�NA"], [1, 0]).astype("bool")
    # OrganiziranKriminal
    df_temp.iloc[:, 9] = df_temp.iloc[:, 9].replace(["ORGANIZIRANA", " "], [1, 0]).astype("bool")
    # MladoletniskaKriminaliteta
    df_temp.iloc[:, 10] = df_temp.iloc[:, 10].replace(["MLADOLETNI�KA", " "], [1, 0]).astype("bool")
    # Poskus
    df_temp.iloc[:, 11] = df_temp.iloc[:, 11].replace(["DA", "NE"], [1, 0]).astype("bool")
    # KriminalisticnaOznacba1
    # KriminalisticnaOznacba2
    # KriminalisticnaOznacba3
    # UporabljenoSredstvo1 TODO
    # UporabljenoSredstvo2 TODO
    # UporabljenoSredstvo3 TODO
    # UporabljenoSredstvo4 TODO
    # UpravnaEnotaStoritve
    # OpisKraja
    # LetoZakljucenegaDokumenta
    # VrstaZakljucenegaDokumenta
    # ZaporednaStevilkaOsebeVKD
    # StarostniRazred TODO
    # TODO

    df_temp.columns = ["ZaporednaStevilkaKD",
                      "MesecStoritve",
                      "UraStoritve",
                      "DanVTednu",
                      "PUStoritveKD",
                      "Povratnik",
                      "OpisKD",
                      "PoglavjeKD",
                      "GospodarskiKriminal",
                      "OrganiziranKriminal",
                      "MladoletniskaKriminaliteta",
                      "Poskus",
                      "KriminalisticnaOznacba1",
                      "KriminalisticnaOznacba2",
                      "KriminalisticnaOznacba3",
                      "UporabljenoSredstvo1",
                      "UporabljenoSredstvo2",
                      "UporabljenoSredstvo3",
                      "UporabljenoSredstvo4",
                       "UpravnaEnotaStoritve",
                       "OpisKraja",
                       "LetoZakljucenegaDokumenta",
                       "VrstaZakljucenegaDokumenta",
                       "ZaporednaStevilkaOsebeVKD",
                       "VrstaOsebe",
                       "StarostniRazred",
                       "Spol",
                       "Drzavljanstvo",
                       "Poskodba",
                       "VplivAlkohola",
                       "VplivMamil",
                       "OrganiziranaZdruzba",
                       "Skoda"
                       ]

    if not df_set:
        df = df_temp
        df_set = True
    else:
        df = df.append(df_temp, ignore_index=True)
df.to_csv("data/kd.csv", index=False)

In [11]:
df = pd.read_csv("data/kd.csv")
df.iloc[:, 0:4]

,ZaporednaStevilkaKD,MesecStoritve,UraStoritve,DanVTednu
0,2009000001,3.2002,0,5
1,2009000001,3.2002,0,5
2,2009000002,6.2002,15,2
3,2009000003,7.2002,17,4
4,2009000003,7.2002,17,4
...,...,...,...,...
1556134,2019061193,11.2019,23,3
1556135,2019061193,11.2019,23,3
1556136,2019061193,11.2019,23,3
1556137,2019061194,9.2016,0,5


## 2. Osnovna vizualizacija